In [33]:
# !pip install transformers
# !pip3 install rouge

In [2]:
import transformers
import pandas as pd
import os
from collections import defaultdict
import re

In [3]:
# df = pd.read_csv('./data/esg_reports/ABT_Abbott/ABT_2021.txt')

In [22]:
reports = {}

for folder in os.listdir('./data/esg_reports'):
    for file in os.listdir(f'./data/esg_reports/{folder}'):
        if file.endswith('2021.csv'):
            df = pd.read_csv(f'./data/esg_reports/{folder}/{file}')
            df = df.rename(columns={'Unnamed: 0': 'Line'})
            # pick out ticker
            ticker = folder.split('_')[0]
            reports[ticker] = df

In [23]:
reports

{'BSX':          Line                                           Sentence  \
 0      Line 1  2021 Performance Report Advancing Science  for...   
 1      Line 2  “The Boston Scientific quality culture inspire...   
 2      Line 3  Arnold worked hard to progress from product bu...   
 3      Line 4  After earning a promotion to quality engineer,...   
 4      Line 5  When doctors showed him the stent they were us...   
 ..        ...                                                ...   
 762  Line 763  All images are the property of Boston Scientif...   
 763  Line 764  All trademarks are the property of their respe...   
 764  Line 765  © 2022 Boston Scientific Corporation or its af...   
 765  Line 766                             All rights reserved.\n   
 766  Line 767                                             PR2021   
 
                                       ESG BERT Topic 1  \
 0    ('Business_Model_Resilience', 0.12671920657157...   
 1    ('Business_Model_Resilience', 0.2305557

In [24]:
df_abt = reports['ABT']
df_abt.head()

,Line,Sentence,ESG BERT Topic 1,ESG BERT Topic 2,ESG BERT Topic 3,Mapped Criteria Topic 1,Mapped Criteria Topic 2,Mapped Criteria Topic 3
0,Line 1,"GLOBAL SUSTAINABILITY REPORT 2021For Abbott,...","('Access_And_Affordability', 0.4256123900413513)","('Employee_Health_And_Safety', 0.1023837178945...","('Product_Design_And_Lifecycle_Management', 0....","('NON-ESG', 0.4256123900413513)","('NON-ESG', 0.10238371789455414)","('NON-ESG', 0.09883648157119751)"
1,Line 2,"In this report, we detail our progress agains...","('Business_Model_Resilience', 0.41427215933799...","('Physical_Impacts_Of_Climate_Change', 0.07623...","('Ecological_Impacts', 0.07013463973999023)","('NON-ESG', 0.41427215933799744)","('NON-ESG', 0.07623419165611267)","('NON-ESG', 0.07013463973999023)"
2,Line 3,The data presented reflect 2021 performance u...,"('Systemic_Risk_Management', 0.4874137043952942)","('Business_Model_Resilience', 0.19010791182518...","('Supply_Chain_Management', 0.03975473344326019)","('NON-ESG', 0.4874137043952942)","('NON-ESG', 0.19010791182518005)","('NON-ESG', 0.03975473344326019)"
3,Line 4,We have aligned our reporting with the requir...,"('Business_Model_Resilience', 0.2894514799118042)","('Access_And_Affordability', 0.11905920505523682)","('Human_Rights_And_Community_Relations', 0.097...","('NON-ESG', 0.2894514799118042)","('NON-ESG', 0.11905920505523682)","('NON-ESG', 0.09785832464694977)"
4,Line 5,We have aligned our reporting with the requir...,('Employee_Engagement_Inclusion_And_Diversity'...,"('Human_Rights_And_Community_Relations', 0.007...","('Data_Security', 0.003554045455530286)","(['Human Capital Development'], 0.952598392963...","('NON-ESG', 0.007480175234377384)","('NON-ESG', 0.003554045455530286)"


In [19]:
# Topics as keys, and a list of all sentences classified as that topic as values

topics_by_company = defaultdict(lambda: defaultdict(lambda: []))

for company in reports:

    sentences_by_topic = defaultdict(lambda: [])

    for index, row in reports[company].iterrows():
        if row['ESG BERT Topic 1'][0] != 'NON-ESG':
            input_string = row['ESG BERT Topic 1']
            match = re.match(r"\('(.+)',\s*(\d+\.\d+)\)", input_string)
            topic = match.group(1)
            number = float(match.group(2))
            sentences_by_topic[topic].append((row['Sentence'], number))
    topics_by_company[company] = sentences_by_topic

# Funkar

In [26]:
# Sort sentences by topic by the number
for company in topics_by_company:
    for topic in topics_by_company[company]:
        sentences_by_topic = topics_by_company[company]
        sentences_by_topic[topic] = sorted(sentences_by_topic[topic], key=lambda x: x[1], reverse=True)

# Funkar

In [27]:
sentences_by_topic

defaultdict(<function __main__.<lambda>()>,
            {'Business_Model_Resilience': [('Shape Tomorrow is our promise to keep our  eyes on the future and create opportunities to move  the business forward.\n',
               0.8105246424674988),
              ('COMPANY PROFILE 2OUR APPROACH TO ESG We take a comprehensive and proactive approach to ESG.\n',
               0.7686639428138733),
              ('Talent One of Zimmer Biomet’s core strategic pillars is our  commitment to being a “Best and Preferred Place  to Work” and we advance that priority through a  comprehensive and integrated approach.\n',
               0.5927088856697083),
              ('Our Approach to ESG .\n', 0.5573867559432983),
              ('Continuous Improvement Zimmer Biomet establishes EHSS goals, objectives  and targets across our global business and strives to  continually improve our performance and publicly report  on our progress.\n',
               0.5261688828468323),
              ('Our Sustainabi

In [7]:
# sentence_list = []
# # Add the 10 first sentences of 'Access_And_Affordability' to the document
# for sentence, number in sentences_by_topic['Access_And_Affordability'][:10]:
#     sentence_list.append(sentence)

# document = " ".join(sentence_list)

In [8]:
# summarizer = transformers.pipeline('summarization')
# summary = summarizer(document, max_length=100, min_length=10, do_sample=False)
# print(summary[0]['summary_text'])

In [47]:
from transformers import BartForConditionalGeneration, BartTokenizer

# Load the pre-trained BART model and tokenizer
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')

output_dict = {}
for company in topics_by_company:
    output_dict[company] = {}
    for topic in topics_by_company[company]:
        sentence_list = []
        sentences_by_topic = topics_by_company[company]
        for sentence, number in sentences_by_topic[topic][:5]:
            # print(topic, sentence, number)
            sentence_list.append(sentence)
        document = " ".join(sentence_list)

        # Encode the input sequence and generate a summary
        input_ids = tokenizer.encode(document, return_tensors='pt')
        summary_ids = model.generate(input_ids, max_length=512, num_beams=4, early_stopping=True)
        summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        output_dict[company][topic] = summary

output_dict

{'BSX': {'Business_Model_Resilience': 'test',
  'Product_Design_And_Lifecycle_Management': 'test',
  'Employee_Health_And_Safety': 'test',
  'Employee_Engagement_Inclusion_And_Diversity': 'test',
  'GHG_Emissions': 'test',
  'Access_And_Affordability': 'test',
  'Human_Rights_And_Community_Relations': 'test',
  'Management_Of_Legal_And_Regulatory_Framework': 'test',
  'Supply_Chain_Management': 'test',
  'Product_Quality_And_Safety': 'test',
  'Ecological_Impacts': 'test',
  'Physical_Impacts_Of_Climate_Change': 'test',
  'Competitive_Behavior': 'test',
  'Energy_Management': 'test',
  'Customer_Welfare': 'test',
  'Systemic_Risk_Management': 'test',
  'Director_Removal': 'test',
  'Labor_Practices': 'test',
  'Business_Ethics': 'test',
  'Customer_Privacy': 'test',
  'Critical_Incident_Risk_Management': 'test',
  'Data_Security': 'test',
  'Waste_And_Hazardous_Materials_Management': 'test',
  'Air_Quality': 'test',
  'Water_And_Wastewater_Management': 'test',
  'Selling_Practices_And_

Evaluation

Summarizer Model POC

In [ ]:
# Define the input sentences
sentences = [
    "The quick brown fox jumps over the lazy dog.",
    "The quick brown fox is a common subject of English pangrams.",
    "A pangram is a sentence that contains every letter of the alphabet.",
    "Pangrams are commonly used in typography and typesetting.",
    "The lazy dog is often used in pangrams because it contains all the letters."
]

# Tokenize the input sentences and combine them into one sequence
input_sequence = "summarize: " + " ".join(sentences)

In [36]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Load the pre-trained T5 model and tokenizer
model = T5ForConditionalGeneration.from_pretrained('t5-small')
tokenizer = T5Tokenizer.from_pretrained('t5-small', model_max_length=512)

# Encode the input sequence and generate a summary
input_ids = tokenizer.encode(input_sequence, return_tensors='pt')
summary_ids = model.generate(input_ids, max_length=50, num_beams=4, early_stopping=True)
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# Print the summary
print(summary)


the quick brown fox is a common subject of English pangrams. a pangram is a sentence that contains every letter of the alphabet.


In [44]:
from transformers import BartForConditionalGeneration, BartTokenizer

# Load the pre-trained BART model and tokenizer
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')

# Encode the input sequence and generate a summary
input_ids = tokenizer.encode(input_sequence, return_tensors='pt')
summary_ids = model.generate(input_ids, max_length=512, num_beams=4, early_stopping=True)
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# Print the summary
print(summary)


A pangram is a sentence that contains every letter of the alphabet. Pangrams are commonly used in typography and typesetting. The lazy dog is often used in pangrams because it contains all the letters. The quick brown fox is a common subject of English panggrams.


Rouge POC

In [49]:
import pandas as pd
from rouge import Rouge

# T5 Model

sentence2 = ["the quick brown fox is a common subject of English pangrams. a pangram is a sentence that contains every letter of the alphabet."]

# Instantiate the ROUGE metric
rouge = Rouge()

# Calculate the ROUGE scores between the two sentences
scores = rouge.get_scores([input_sequence], sentence2)

# Print the ROUGE scores
print(scores)


[{'rouge-1': {'r': 1.0, 'p': 0.4864864864864865, 'f': 0.6545454501421489}, 'rouge-2': {'r': 0.8571428571428571, 'p': 0.3829787234042553, 'f': 0.5294117604368512}, 'rouge-l': {'r': 1.0, 'p': 0.4864864864864865, 'f': 0.6545454501421489}}]


In [50]:
# Facebook model

sentence2 = ["A pangram is a sentence that contains every letter of the alphabet. Pangrams are commonly used in typography and typesetting. The lazy dog is often used in pangrams because it contains all the letters. The quick brown fox is a common subject of English panggrams."]

# Instantiate the ROUGE metric
rouge = Rouge()

# Calculate the ROUGE scores between the two sentences
scores = rouge.get_scores([input_sequence], sentence2)

# Print the ROUGE scores
print(scores)


[{'rouge-1': {'r': 0.9722222222222222, 'p': 0.9459459459459459, 'f': 0.9589041045899794}, 'rouge-2': {'r': 0.9523809523809523, 'p': 0.851063829787234, 'f': 0.8988763995101628}, 'rouge-l': {'r': 0.9722222222222222, 'p': 0.9459459459459459, 'f': 0.9589041045899794}}]
